In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

In [2]:
train_data = datasets.MNIST(root='', train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST(root='', train=False, transform=transforms.ToTensor(), download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



In [8]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [9]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

input_size = 28*28
hidden_size = 512
num_classes = 10
model = MLP(input_size, hidden_size, num_classes)

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [16]:
# 训练网络
num_epochs = 10 # 训练轮数
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将images转成向量
        images = images.reshape(-1, 28 * 28)
        # 将数据送入网络中
        outputs = model(images)
        # 计算损失
        loss = criterion(outputs, labels)
        # 清零梯度
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        if (i+1) % 100 ==0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/600], Loss: 0.0029
Epoch [1/10], Step [200/600], Loss: 0.0406
Epoch [1/10], Step [300/600], Loss: 0.0004
Epoch [1/10], Step [400/600], Loss: 0.0200
Epoch [1/10], Step [500/600], Loss: 0.0034
Epoch [1/10], Step [600/600], Loss: 0.0084
Epoch [2/10], Step [100/600], Loss: 0.0001
Epoch [2/10], Step [200/600], Loss: 0.0001
Epoch [2/10], Step [300/600], Loss: 0.0018
Epoch [2/10], Step [400/600], Loss: 0.0669
Epoch [2/10], Step [500/600], Loss: 0.0062
Epoch [2/10], Step [600/600], Loss: 0.0158
Epoch [3/10], Step [100/600], Loss: 0.0560
Epoch [3/10], Step [200/600], Loss: 0.0018
Epoch [3/10], Step [300/600], Loss: 0.0002
Epoch [3/10], Step [400/600], Loss: 0.0034
Epoch [3/10], Step [500/600], Loss: 0.0006
Epoch [3/10], Step [600/600], Loss: 0.0157
Epoch [4/10], Step [100/600], Loss: 0.0005
Epoch [4/10], Step [200/600], Loss: 0.0004
Epoch [4/10], Step [300/600], Loss: 0.0197
Epoch [4/10], Step [400/600], Loss: 0.0004
Epoch [4/10], Step [500/600], Loss: 0.0002
Epoch [4/10

In [17]:
# 测试网络
with torch.no_grad():
    correct = 0
    total = 0
    # 从test_loader中循环获取测试数据
    for images, labels in test_loader:
        # 将images转化为晨
        images = images.reshape(-1, 28 * 28)
        # 将数据送入网络
        outputs = model(images)
        # 取出最大值对应的索引，即预测值
        _, predicted = torch.max(outputs.data, 1)
        # 计算label数量
        total += labels.size(0)
        # 预测值与labels相比较，获取预测正确的数量
        correct += (predicted == labels).sum().item()
    # 打印最终的准确率
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 98.48 %


In [18]:
torch.save(model,"handwrite.pkl")